In [65]:
import io
from pathlib import Path

import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

from astropy import units as u
from astropy import table
from astropy.io import votable

import pyvo as vo

from tqdm.notebook import tqdm

from multiprocess import Pool

from dill import dump, load

In [4]:
import warnings
from astroquery.exceptions import NoResultsWarning
warnings.simplefilter('ignore', NoResultsWarning)
warnings.simplefilter('ignore', u.UnitsWarning) # comes up in the parsing of some heasarc tables but is just a "discouraged by FITS standard" issue

In [5]:
import SAGA
saga = SAGA.QuickStart('/Users/etollerud/Dropbox/', '/../')

SAGA `shared_dir` set to /Users/etollerud/Dropbox/
SAGA `local_dir`  set to /../


In [6]:
hosts = saga.host_catalog.load()

In [26]:
hosts_vot = votable.from_table(hosts['HOSTID', 'RA','DEC'])
hosts_vot.get_first_table()

<VOTable length=300>
  HOSTID        RA         DEC    
  str10      float64     float64  
---------- ----------- -----------
 nsa126115    0.995382  20.7523559
   pgc1952    8.008776 -64.2533874
   pgc2052    8.564589 -27.8036183
 nsa127226    8.694936  -8.3964957
   pgc2778   11.864958 -31.4216944
   pgc3089   13.176354 -31.2058056
       ...         ...         ...
  pgc72009 354.7378755 -12.9610833
  pgc72060 354.9742875 -12.2927807
 nsa169525  356.065458  10.7670833
  pgc72444 356.9657175 -30.5220634
 nsa153017 359.8561875  20.7499195
 pgc771919   46.033425 -26.0696911
pgc3806285 200.0691045 -24.4462083

# Match using HEASARC's PyVO interface

According to Tess Jaffe, the astroquery interface is using an older not-well-supported approach, and is to be updated whenever HEASARC can get the resources to do it.

In [7]:
import pyvo

In [10]:
tap_services = pyvo.regsearch(servicetype='tap', keywords=['heasarc'])
assert len(tap_services)==1
heasarc_tap = tap_services[0]

tables_dct = dict(heasarc_tap.service.tables.items())

In [115]:
radius = 12*u.arcmin

In [117]:
radius.to(u.deg).value

0.2

In [120]:
def get_matches_for_catalog(cat, radius, table, idcol='HOSTID'):
    deg = radius.to(u.deg).value
    query=f"""
    SELECT *, mt.{idcol}
    FROM chanmaster cat, tap_upload.mysources mt
    WHERE 
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,{deg}))=1
    ORDER by cat.ra
    """
    
    vot = votable.from_table(cat[idcol, 'RA', 'DEC'])
    bio = io.BytesIO()
    vot.to_xml(bio)
    bio.seek(0)
    
    res = heasarc_tap.service.run_async(query, uploads={'mysources': bio})
    return res.to_table()

get_matches_for_catalog(hosts, 15*u.arcmin, 'chanmaster')

chanmaster___row,chanmaster_obsid,chanmaster_status,chanmaster_name,chanmaster_ra,chanmaster_dec,chanmaster_lii,chanmaster_bii,chanmaster_time,chanmaster_detector,chanmaster_grating,chanmaster_exposure,chanmaster_type,chanmaster_pi,chanmaster_cycle,chanmaster_proposal,chanmaster_public_date,chanmaster_sequence_number,chanmaster_data_mode,chanmaster_category,chanmaster_class,chanmaster___x_ra_dec,chanmaster___y_ra_dec,chanmaster___z_ra_dec,hostid
,,,,deg,deg,deg,deg,d,,,s,,,,,d,,,,,,,,
int32,int32,object,object,float64,float64,float64,float64,float64,object,object,float64,object,object,int16,object,int32,int32,object,object,int16,float64,float64,float64,object
9685,23571,archived,SN 2009em,8.68554,-8.39933,110.24683,-70.85875,59115.954849537,ACIS-S,NONE,12080,GO,Patnaude,22,22500259,59481,503246,TE_0057A,"SN, SNR AND ISOLATED NS",3200,0.149391603204477,0.977929075843426,-0.14607146029262,nsa127226
9688,27644,archived,SN 2022jli,8.69037,-8.38671,110.26870,-70.84733,59961.4469907407,ACIS-S,NONE,10060,DDT,Chen,23,23508880,59962,503448,TE_007BC,"SN, SNR AND ISOLATED NS",3200,0.149478899171775,0.977948259426569,-0.145853558705086,nsa127226
9689,27646,archived,SN 2022jli,8.69037,-8.38671,110.26870,-70.84733,59969.882962963,ACIS-S,NONE,10060,DDT,Chen,23,23508880,59970,503450,TE_007BC,"SN, SNR AND ISOLATED NS",3200,0.149478899171775,0.977948259426569,-0.145853558705086,nsa127226
9690,27645,archived,SN 2022jli,8.69037,-8.38671,110.26870,-70.84733,59965.9437962963,ACIS-S,NONE,10060,DDT,Chen,23,23508880,59969,503449,TE_007BC,"SN, SNR AND ISOLATED NS",3200,0.149478899171775,0.977948259426569,-0.145853558705086,nsa127226
5179,16352,archived,NGC289,13.17667,-31.20583,299.12257,-85.91303,56881.5004398148,ACIS-I,NONE,50090,GTO,Garmire,15,15610062,57247,601152,TE_006E6,NORMAL GALAXIES,6000,0.194972050737596,0.832792734473645,-0.518114042310268,pgc3089
13871,4408,archived,NGC 821,32.08750,10.99472,151.55478,-47.55719,52604.7187152778,ACIS-S,NONE,25270,GO,Fabbiano,4,04700486,52984,700685,TE_005A0,ACTIVE GALAXIES AND QUASARS,7000,0.521463199181968,0.831686583139149,0.190718534299197,pgc8160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6444,11344,archived,NGC7507,348.02917,-28.54061,23.44014,-68.03611,55471.1952893519,ACIS-S,NONE,7350,GO,Gallo,11,11620915,55483,600863,TE_00600,NORMAL GALAXIES,6000,-0.182208495439119,0.859374701084884,-0.477781526772518,pgc70676


In [121]:
mission_to_rad = {'xmmmaster': 15*u.arcmin, # " telescope's field of view (FOV) of 30 arcmin and in th"
                  'chanmaster':8*u.arcmin, # this is ~ half ASIC's FOV
                  'rosmaster':2.5*u.deg,  #"5 degree diameter"
                  'rassmaster':2.5*u.deg,  #"5 degree diameter"
                }

matches = {}
for tab in tqdm(mission_to_rad):
    matches[tab] = get_matches_for_catalog(hosts, mission_to_rad[tab], tab)

  0%|          | 0/4 [00:00<?, ?it/s]